In [176]:
import numpy as np
import csv
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses

items_total = 100 #10000
cats_total = 48 # 1174 para 10K | 48 para 100
split = 0.8
train_total = int(items_total * split)
test_total = int(items_total - train_total)

print (">> Train total: "+str(train_total))
print (">> Test total: "+str(test_total))

# data arrays
train_data = []
train_labels = []
validation_data = []
validation_labels = []
test_data = []
test_labels = []

# Creamos dir con las categorias
newpath = '/categories' 
if not os.path.exists(newpath):
    os.makedirs(newpath)

# Folders de train y test
newpath_train = '/categories/train'
newpath_test = '/categories/test'

# Obtenemos train y validation
with open('/Users/raulrodriguez_demarque/demarque/market/categories_100.csv') as csvfile:
  reader = csv.reader(csvfile, delimiter=',')
  step = 0
  for row in reader:
    if step < train_total:
      train_data.append(row[2].replace('\n', ' ').replace('\r', ''))
      train_labels.append(row[1])
    else:
      validation_data.append(row[2].replace('\n', ' ').replace('\r', ''))
      validation_labels.append(row[1])
    step += 1

# Ahora obtenemos el test
with open('/Users/raulrodriguez_demarque/demarque/market/categories_100.csv') as csvfile_test:
    reader_test = csv.reader(csvfile_test, delimiter=',')
    for row_test in reader_test:
      test_data.append(row[2].replace('\n', ' ').replace('\r', ' '))
      test_labels.append(row[1])

print(">> Training data length: "+str(len(train_data)))
print(">> Validation data length: "+str(len(validation_data)))
print(">> Test data length: "+str(len(test_data)))
print(train_labels[0])
print(train_data[0])

>> Train total: 80
>> Test total: 20
>> Training data length: 80
>> Validation data length: 20
>> Test data length: 100
FBFIC009030
THE HARD-BITTEN CHAMPION OF BRITISH HEROIC FANTASY' - Joe Abercrombie 'HEROISM AND HEARTBREAK . . . GEMMELL IS ADRENALINE WITH SOUL' - Brent Weeks It is three hundred years since the world toppled on its axis and  civilisation was destroyed. In this savagely reshaped world ruled by  brigands and war-makers, a rider seeks a lost city. Pursuing a dream to  calm the violence in his soul, Jon Shannow, the brigand slayer, desires  only peace. But from the Plague Lands emerges a fresh terror.The  Lord of the Pit and his hellborn army seek to plunge mankind into a new demonic era. Seemingly invincible, they make a fatal mistake: they take  Shannow's woman for blood sacrifice. And find themselves facing the  deadliest warrior of the new age. Jon Shannow - the Jerusalem Man.Novels by David GemmellThe Drenai seriesLegend The King Beyond the GateWaylanderQuest For Lo

In [177]:
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation),
                                  '')
max_features = 10000
sequence_length = 250

vectorize_layer = layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

In [212]:
# Make a few datasets

#Convertimos nuestros datos a numpy arrays DO I NEED THIS?
np_train_labels = np.array(train_labels)
np_train_data = np.array(train_data)

np_validation_labels = np.array(validation_labels)
np_validation_data = np.array(validation_data)

np_test_labels = np.array(test_labels)
np_test_data = np.array(test_data)

# Este tampoco adapta pero hay que probarlo XD
raw_train_ds = tf.data.Dataset.from_tensor_slices(
  (np_train_data, np_train_labels)
)
raw_validation_ds = tf.data.Dataset.from_tensor_slices(
    (np_validation_data, np_validation_labels)
)
raw_test_ds = tf.data.Dataset.from_tensor_slices(
    (np_test_data, np_test_labels)
)


# no veo la diferencia de esto contra hacerlo directo (sin tf.convert_to_tensor)
print(">> raw_train_ds: "+str(raw_train_ds))



for index, item in enumerate(raw_train_ds):  # only take first element of dataset
    print(">> ITEM--------------------------------")
    # item[0].ensure_shape(None,)
    # item[1].ensure_shape(None,)
    print(item[0])
    print(item[1])
    if index == 2:
        break

>> raw_train_ds: <_TensorSliceDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.string, name=None))>
>> ITEM--------------------------------
tf.Tensor(b"THE HARD-BITTEN CHAMPION OF BRITISH HEROIC FANTASY' - Joe Abercrombie 'HEROISM AND HEARTBREAK . . . GEMMELL IS ADRENALINE WITH SOUL' - Brent Weeks It is three hundred years since the world toppled on its axis and  civilisation was destroyed. In this savagely reshaped world ruled by  brigands and war-makers, a rider seeks a lost city. Pursuing a dream to  calm the violence in his soul, Jon Shannow, the brigand slayer, desires  only peace. But from the Plague Lands emerges a fresh terror.The  Lord of the Pit and his hellborn army seek to plunge mankind into a new demonic era. Seemingly invincible, they make a fatal mistake: they take  Shannow's woman for blood sacrifice. And find themselves facing the  deadliest warrior of the new age. Jon Shannow - the Jerusalem Man.Novels by David Gem

In [207]:
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation),
                                  '')

In [208]:
max_features = 10000
sequence_length = 250

vectorize_layer = layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    encoding="utf-8",
    output_sequence_length=sequence_length)

In [233]:
print(">> raw_train_ds: "+str(raw_train_ds))
tf.config.run_functions_eagerly(True)

#test, maybe remove
# batch_size = 32
# raw_train_ds = raw_train_ds.batch(batch_size)

# Make a text-only dataset (without labels), then call adapt
train_text = raw_train_ds.map(lambda x, y: x)

# Test
texts_only = []
for index, item in enumerate(train_text):
    # estoy pensando que quiero adaptar un tensor, no un string y ese puede ser el problema
    # print(index)
    # print(item.numpy().decode('ascii')) # esto no sirva para obtener el string
    # if index == 2:
    #     break
    texts_only.append(item.numpy().decode('utf-8'))

vectorize_layer.adapt(texts_only)

# print(">> train_text: "+str(train_text))
# vectorize_layer.adapt(train_text)
# # print(">> train_text: "+str(train_text)) # esto no funciona
# # print(">> train_text: "+str(train_data))

# # Esto funciona pero me da built=False
# # vectorize_layer.adapt(train_data)
print(">> vectorize_layer adapted: "+str(vectorize_layer))

>> raw_train_ds: <_TensorSliceDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.string, name=None))>
>> vectorize_layer adapted: <TextVectorization name=text_vectorization_28, built=False>


2024-04-25 05:47:02.583523: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [234]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text), label

In [241]:
# retrieve a batch (of 32 reviews and labels) from the dataset
text_batch, label_batch = next(iter(raw_train_ds))
print(">> text_batch[0]: "+str(text_batch[0]))
print(">> label_batch[0]: "+str(label_batch[0]))

first_review, first_label = text_batch[0], label_batch[0]
print("Review", first_review)
print("Label", raw_train_ds.class_names[first_label])
print("Vectorized review", vectorize_text(first_review, first_label))

2024-04-25 06:01:23.896124: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at strided_slice_op.cc:117 : INVALID_ARGUMENT: Attempting to slice scalar input.
2024-04-25 06:01:23.896144: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Attempting to slice scalar input.


InvalidArgumentError: {{function_node __wrapped__StridedSlice_device_/job:localhost/replica:0/task:0/device:CPU:0}} Attempting to slice scalar input. [Op:StridedSlice] name: strided_slice/

In [236]:
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_validation_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

In [237]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [238]:
embedding_dim = 16

model = tf.keras.Sequential([
  layers.Embedding(max_features, embedding_dim),
  layers.Dropout(0.2),
  layers.GlobalAveragePooling1D(),
  layers.Dropout(0.2),
  # layers.Dense(1, activation='sigmoid')
  layers.Dense(128, activation='relu'),
  layers.Dense(cats_total)
])

model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_3      │ ?                      │   0 (unbuilt) │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [239]:
model.compile(
    # loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer='adam',
    metrics=['accuracy']
)

In [240]:
loss, accuracy = model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

2024-04-25 05:47:44.488438: W tensorflow/core/framework/op_kernel.cc:1816] OP_REQUIRES failed at cast_op.cc:122 : UNIMPLEMENTED: Cast string to float is not supported
2024-04-25 05:47:44.488462: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: UNIMPLEMENTED: Cast string to float is not supported


UnimplementedError: {{function_node __wrapped__Cast_device_/job:localhost/replica:0/task:0/device:CPU:0}} Cast string to float is not supported [Op:Cast] name: 